In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import math
import seaborn as sns

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from models import control_ma,control_tapper,complex,complex_long

In [4]:
control_ma_info = {"name":"control_ma","pipe":control_ma.Control}
control_tapper_info = {"name":"control_tapper","pipe":control_tapper.Control}
complex_info = {"name":"complex","pipe":complex.ComplexModel}

In [5]:
def apply_fpc(input_fids,pipe,t,starting_sp=512):
    fids = input_fids.copy()
    # 1752-728

    freq_pred_spec = np.fft.fftshift(np.fft.ifft(fids,axis=1),axes=1)
    freq_pred_spec = freq_pred_spec[:,starting_sp:(starting_sp+1024),:,:]

    freq_results = pipe.predict_freq(freq_pred_spec).reshape(fids.shape[0],1,1,-1)

    fids = fids*np.exp(1j*freq_results*2*math.pi*t.reshape(1,-1,1,1))

    phase_pred_spec = np.fft.fftshift(np.fft.ifft(fids,axis=1),axes=1)
    phase_pred_spec = phase_pred_spec[:,starting_sp:(starting_sp+1024),:,:]

    phase_results = pipe.predict_phase(phase_pred_spec).reshape(fids.shape[0],1,1,-1)

    fids = fids*np.exp(1j*phase_results*math.pi/180)

    return fids,freq_results,phase_results



In [6]:
# calculate fwhm of peak in specific range (nor sure 100% correct)
def calc_fwhms(spectra,ppm,ppm_min,ppm_max):
    freqInt = np.where((ppm <= ppm_max) & (ppm >= ppm_min))
    
    medians = np.median(spectra,axis=1)
    
    range_spectrum = spectra[:,freqInt[0][0]:freqInt[0][-1]]
    range_ppm = ppm[freqInt[0][0]:freqInt[0][-1]]

    maxes = range_spectrum.max(axis=1)

    over_half_range = (range_spectrum-medians.reshape(-1,1)>=((maxes-medians)/2).reshape(-1,1))

    output_fwhms=np.zeros(spectra.shape[0])

    for i in range(spectra.shape[0]):
        freqInt2 = np.where(over_half_range[i]==True)
        if(freqInt2[0].shape[0]==0):
            max_ppm_ind = 0
            min_ppm_ind = freqInt2.shape[0]-1
        elif (freqInt2[0].shape[0]==1):
            max_ppm_ind = max(freqInt2[0][0]-1,0)
            min_ppm_ind = min(freqInt2[0][0]+1,range_ppm.shape[0]-1)
        else:
            #print(freqInt2)
            max_ppm_ind = max(freqInt2[0][0]-1,0)
            min_ppm_ind = min(freqInt2[0][-1]+1,range_ppm.shape[0]-1) #  we need to define if we use the point right before or after the max_ppm range

        min_ppm = range_ppm[min_ppm_ind]
        max_ppm = range_ppm[max_ppm_ind] 
        output_fwhms[i]=(max_ppm-min_ppm)
    
    return output_fwhms

    

In [7]:
## calculate SNR of given ppm range
def calc_snr(spectra,ppm,ppm_min,ppm_max):

    output_snrs = np.zeros(spectra.shape[0])

    for i in range(spectra.shape[0]):
        indClose, indFar = np.amax(np.where(ppm >= ppm_min)), np.amin(np.where(ppm <= ppm_max))
        dt_indClose, dt_indFar = np.amax(np.where(ppm >= 10.0)), np.amin(np.where(ppm <= 12))
        
        max_peak = np.amax(spectra[i,indFar:indClose])
        # Std Dev of Noise
        dt = np.polyfit(ppm[dt_indFar:dt_indClose], ppm[dt_indFar:dt_indClose], 2)
        
        sizeFreq = ppm[dt_indFar:dt_indClose].shape[0]
        #stdev_Man = np.real(spectra[i,dt_indFar:dt_indClose]).std()#= np.sqrt(np.sum(np.square(np.real(i_spec[dt_indFar:dt_indClose] - 0*np.polyval(dt, i_ppm[dt_indFar:dt_indClose])))) / (sizeFreq - 1))
        stdev_Man = np.sqrt(np.sum(np.square(np.real(spectra[i,dt_indFar:dt_indClose] - np.polyval(dt, ppm[dt_indFar:dt_indClose])))) / (sizeFreq - 1))
        snr = np.real(max_peak) / (2 * stdev_Man)

        output_snrs[i]=snr  
    return output_snrs     

In [ ]:
## Metrics for
training_run=1

scans = []
model_snrs=[]
cho_stds = []
models=[]
on_cr_fwhms=[]
off_cr_fwhms=[]
gaba_fwhms=[]
on_cr_snrs=[]
off_cr_snrs=[]
gaba_snrs=[]


transient_scans=[]
transients=[]
freq_preds=[]
freq_truths=[]
phase_preds=[]
phase_truths=[]
water_supressions=[]
on_offs=[]
transient_model_snrs=[]
transient_models=[]

for model_snr in ["2_5"]:#,"5","10","x"]:
    for wsup in ["unsup"]:#["sup","unsup"]:
        off_fids = np.load(f"root/data/noisy_datasets/test_datasets/off_fids_{wsup}_{model_snr}.npy")
        on_fids = np.load(f"root/data/noisy_datasets/test_datasets/on_fids_{wsup}_{model_snr}.npy")
        ppm = np.load(f"root/data/noisy_datasets/test_datasets/ppm_{wsup}_{model_snr}.npy")
        t = np.load(f"root/data/noisy_datasets/test_datasets/t_{wsup}_{model_snr}.npy")
        
        y_freq = np.load(f"root/data/noisy_datasets/test_datasets/y_dataFreq_{wsup}_{model_snr}.npy")
        y_phase = np.load(f"root/data/noisy_datasets/test_datasets/y_dataPhase_{wsup}_{model_snr}.npy")

        raw_off_fids = np.expand_dims(off_fids,axis=2)
        raw_on_fids = np.expand_dims(on_fids,axis=2)

        raw_fids = np.concatenate([raw_off_fids,raw_on_fids],axis=2)


        #print(ppm.shape)
        #print(t.shape)
        #print(raw_fids.shape)
        #print(y_freq.shape)



        for model in [complex_info,control_ma_info,control_tapper_info]:

            pipe_off = model["pipe"]()
            pipe_off.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            off_fids,off_freq_pred,off_phase_pred = apply_fpc(raw_fids[:,:,:1,:],pipe_off,t,512)

            pipe_on = model["pipe"]()
            pipe_on.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            on_fids,on_freq_pred,on_phase_pred = apply_fpc(raw_fids[:,:,1:,:],pipe_on,t,512)


            off_y_freq = y_freq[:,:,0,:].flatten()
            on_y_freq = y_freq[:,:,1,:].flatten()
            off_y_phase = y_phase[:,:,0,:].flatten()
            on_y_phase = y_phase[:,:,1,:].flatten()

            transients+=list(range(1,161))*2*off_fids.shape[0]
            transient_scans+=list(np.repeat(np.arange(1,off_fids.shape[0]+1),320))
            freq_preds += list(off_freq_pred.flatten()) + list(on_freq_pred.flatten())
            phase_preds += list(off_phase_pred.flatten()) + list(on_phase_pred.flatten())
            freq_truths+= list(off_y_freq) + list(on_y_freq)
            phase_truths+= list(off_y_phase) + list(on_y_phase)

            if wsup=="sup":
                water_supression = True
            else:
                water_supression = False

            water_supressions+=[water_supression]*320*on_fids.shape[0]
            on_offs+=list(np.repeat(np.array([False,True]),160*on_fids.shape[0]))
            transient_model_snrs+=[model_snr]*320*on_fids.shape[0]
            transient_models+=[model["name"]]*320*on_fids.shape[0]
            
            #second part

            off_specs = np.real(np.fft.fftshift(np.fft.ifft(off_fids,axis=1),axes=1).mean(axis=3))
            on_specs = np.real(np.fft.fftshift(np.fft.ifft(on_fids,axis=1),axes=1).mean(axis=3))
            diff_specs = on_specs-off_specs

            ###calculate choline artifact std
            freqInt = np.where((ppm <= 3.285) & (ppm >= 3.15))
            cho_std = diff_specs[:,freqInt[0][0]:freqInt[0][-1],0].reshape(diff_specs.shape[0],-1).std(axis=1)

            ###calculate linewidths
            cr_on_fwhm = calc_fwhms(np.squeeze(on_specs),ppm,2.8,3.2)
            cr_off_fwhm = calc_fwhms(np.squeeze(off_specs),ppm,2.8,3.2)
            gaba_fwhm = calc_fwhms(np.squeeze(diff_specs),ppm,2.8,3.2)

            ###
            cr_on_snr = calc_snr(np.squeeze(on_specs),ppm,2.8,3.2)
            cr_off_snr = calc_snr(np.squeeze(off_specs),ppm,2.8,3.2)
            gaba_snr = calc_snr(np.squeeze(diff_specs),ppm,2.8,3.2)
        

            size = cho_std.shape[0]
            scans += list(range(1,size+1))
            model_snrs+=[model_snr]*size
            cho_stds+=list(cho_std)
            on_cr_fwhms+=list(cr_on_fwhm)
            off_cr_fwhms+=list(cr_off_fwhm)
            gaba_fwhms+=list(gaba_fwhm)
            on_cr_snrs+=list(cr_on_snr)
            off_cr_snrs+=list(cr_off_snr)
            gaba_snrs+=list(gaba_snr)
            models+=[model["name"]]*size  




metrics_df = pd.DataFrame({
    "scan":scans,
    "model_snr":model_snrs,
    "model":models,
    "cho_std":cho_stds,
    "on_cr_fwhm":on_cr_fwhms,
    "off_cr_fwhm":off_cr_fwhms,
    "gaba_fwhm":gaba_fwhms,
    "on_cr_snr":on_cr_snrs,
    "off_cr_snr":off_cr_snrs,
    "gaba_snr":gaba_snrs
})

transient_metrics_df = pd.DataFrame({
    "transient":transients,
    "scan":transient_scans,
    "model_snr":transient_model_snrs,
    "model":transient_models,
    "water_supression":water_supressions,
    "on_off":on_offs,
    "freq_pred":freq_preds,
    "phase_pred":phase_preds,
    "true_freq":freq_truths,
    "true_phase":phase_truths
})

In [39]:
## Metrics for
training_run=1

scans = []
model_snrs=[]
cho_stds = []
models=[]
on_cr_fwhms=[]
off_cr_fwhms=[]
gaba_fwhms=[]
on_cr_snrs=[]
off_cr_snrs=[]
gaba_snrs=[]

scan_choice=1
scan_count=1
transient_choice=1
transient_count=2

transient_scans=[]
transients=[]
freq_preds=[]
freq_truths=[]
phase_preds=[]
phase_truths=[]
water_supressions=[]
on_offs=[]
transient_model_snrs=[]
transient_models=[]

for model_snr in ["10"]:#,"5","10","x"]:
    for wsup in ["unsup"]:#["sup","unsup"]:
        off_fids = np.load(f"root/data/noisy_datasets/test_datasets/off_fids_{wsup}_{model_snr}.npy")[scan_choice:scan_choice+scan_count,:,transient_choice:transient_choice+transient_count]
        on_fids = np.load(f"root/data/noisy_datasets/test_datasets/on_fids_{wsup}_{model_snr}.npy")[scan_choice:scan_choice+scan_count,:,transient_choice:transient_choice+transient_count]
        ppm = np.load(f"root/data/noisy_datasets/test_datasets/ppm_{wsup}_{model_snr}.npy")
        t = np.load(f"root/data/noisy_datasets/test_datasets/t_{wsup}_{model_snr}.npy")
        
        y_freq = np.load(f"root/data/noisy_datasets/test_datasets/y_dataFreq_{wsup}_{model_snr}.npy")[scan_choice:scan_choice+scan_count,:,:,transient_choice:transient_choice+transient_count]
        y_phase = np.load(f"root/data/noisy_datasets/test_datasets/y_dataPhase_{wsup}_{model_snr}.npy")[scan_choice:scan_choice+scan_count,:,:,transient_choice:transient_choice+transient_count]

        raw_off_fids = np.expand_dims(off_fids,axis=2)
        raw_on_fids = np.expand_dims(on_fids,axis=2)

        raw_fids = np.concatenate([raw_off_fids,raw_on_fids],axis=2)


        #print(ppm.shape)
        #print(t.shape)
        #print(raw_fids.shape)
        #print(y_freq.shape)



        for model in [complex_info]:#,control_ma_info,control_tapper_info]:

            pipe_off = model["pipe"]()
            pipe_off.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            off_fids,off_freq_pred,off_phase_pred = apply_fpc(raw_fids[:,:,:1,:],pipe_off,t,512)

            pipe_on = model["pipe"]()
            pipe_on.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            on_fids,on_freq_pred,on_phase_pred = apply_fpc(raw_fids[:,:,1:,:],pipe_on,t,512)


            off_y_freq = y_freq[:,:,0,:].flatten()
            on_y_freq = y_freq[:,:,1,:].flatten()
            off_y_phase = y_phase[:,:,0,:].flatten()
            on_y_phase = y_phase[:,:,1,:].flatten()

            transients+=list(range(1,transient_count+1))*2*off_fids.shape[0]
            transient_scans+=list(np.repeat(np.arange(1,off_fids.shape[0]+1),2*transient_count))
            freq_preds += list(off_freq_pred.flatten()) + list(on_freq_pred.flatten())
            phase_preds += list(off_phase_pred.flatten()) + list(on_phase_pred.flatten())
            freq_truths+= list(off_y_freq) + list(on_y_freq)
            phase_truths+= list(off_y_phase) + list(on_y_phase)

            if wsup=="sup":
                water_supression = True
            else:
                water_supression = False

            water_supressions+=[water_supression]*2*transient_count*on_fids.shape[0]
            on_offs+=list(np.repeat(np.array([False,True]),transient_count*on_fids.shape[0]))
            transient_model_snrs+=[model_snr]*2*transient_count*on_fids.shape[0]
            transient_models+=[model["name"]]*2*transient_count*on_fids.shape[0]
            
            #second part

            off_specs = np.real(np.fft.fftshift(np.fft.ifft(off_fids,axis=1),axes=1).mean(axis=3))
            on_specs = np.real(np.fft.fftshift(np.fft.ifft(on_fids,axis=1),axes=1).mean(axis=3))
            diff_specs = on_specs-off_specs

            ###calculate choline artifact std
            freqInt = np.where((ppm <= 3.285) & (ppm >= 3.15))
            cho_std = diff_specs[:,freqInt[0][0]:freqInt[0][-1],0].reshape(diff_specs.shape[0],-1).std(axis=1)

            ###calculate linewidths
            cr_on_fwhm = np.array([0])#calc_fwhms(np.squeeze(on_specs),ppm,2.8,3.2)
            cr_off_fwhm = np.array([0])#calc_fwhms(np.squeeze(off_specs),ppm,2.8,3.2)
            gaba_fwhm = np.array([0])#calc_fwhms(np.squeeze(diff_specs),ppm,2.8,3.2)

            ###
            cr_on_snr = np.array([0])#calc_snr(np.squeeze(on_specs),ppm,2.8,3.2)
            cr_off_snr = np.array([0])#calc_snr(np.squeeze(off_specs),ppm,2.8,3.2)
            gaba_snr = np.array([0])#calc_snr(np.squeeze(diff_specs),ppm,2.8,3.2)
        

            size = cho_std.shape[0]
            scans += list(range(1,size+1))
            model_snrs+=[model_snr]*size
            cho_stds+=list(cho_std)
            on_cr_fwhms+=list(cr_on_fwhm)
            off_cr_fwhms+=list(cr_off_fwhm)
            gaba_fwhms+=list(gaba_fwhm)
            on_cr_snrs+=list(cr_on_snr)
            off_cr_snrs+=list(cr_off_snr)
            gaba_snrs+=list(gaba_snr)
            models+=[model["name"]]*size  




metrics_df = pd.DataFrame({
    "scan":scans,
    "model_snr":model_snrs,
    "model":models,
    "cho_std":cho_stds,
    "on_cr_fwhm":on_cr_fwhms,
    "off_cr_fwhm":off_cr_fwhms,
    "gaba_fwhm":gaba_fwhms,
    "on_cr_snr":on_cr_snrs,
    "off_cr_snr":off_cr_snrs,
    "gaba_snr":gaba_snrs
})



transient_metrics_df = pd.DataFrame({
    "transient":transients,
    "scan":transient_scans,
    "model_snr":transient_model_snrs,
    "model":transient_models,
    "water_supression":water_supressions,
    "on_off":on_offs,
    "freq_pred":freq_preds,
    "phase_pred":phase_preds,
    "true_freq":freq_truths,
    "true_phase":phase_truths
})

In [40]:
print(transient_metrics_df[["model","water_supression","transient","freq_pred","true_freq"]])

     model  water_supression  transient  freq_pred  true_freq
0  complex             False          1   0.718576  -7.942582
1  complex             False          2   8.283858   2.778715
2  complex             False          1   1.787907 -20.287141
3  complex             False          2  -3.713300  -5.878928


In [19]:
transient_metrics_df["freq_error"]=transient_metrics_df["freq_pred"]-transient_metrics_df["true_freq"]
transient_metrics_df["phase_error"]=transient_metrics_df["phase_pred"]-transient_metrics_df["true_phase"]

transient_metrics_df["abs_freq_error"] = np.abs(transient_metrics_df["freq_error"])
transient_metrics_df["abs_phase_error"] = np.abs(transient_metrics_df["phase_error"])

In [20]:
print(transient_metrics_df[(transient_metrics_df["on_off"]==False)&(transient_metrics_df["transient"]==1)].groupby(["model_snr","model"]).agg({"abs_freq_error":"mean","abs_phase_error":"mean"}))

                          abs_freq_error  abs_phase_error
model_snr model                                          
2_5       complex               7.304265         4.584745
          control_ma            7.188296         3.643619
          control_tapper        7.052556         2.691425


In [11]:
i_df = transient_metrics_df[(transient_metrics_df["on_off"]==False)&(transient_metrics_df["model"]=="complex")]
i_f_p = i_df["freq_pred"].values
i_t_f = i_df["true_freq"].values

i_f_p.sort()
i_t_f.sort()
diff = np.abs(i_f_p-i_t_f)
print(diff.mean())

2.376850010620118


In [17]:
t = np.load("root/data/in_vivo_datasets/t.npy")
ppm = np.load("root/data/in_vivo_datasets/ppm.npy")


## Metrics for
training_run=1

scans = []
model_snrs=[]
cho_stds = []
models=[]
on_cr_fwhms=[]
off_cr_fwhms=[]
gaba_fwhms=[]
on_cr_snrs=[]
off_cr_snrs=[]
gaba_snrs=[]


transient_scans=[]
transients=[]
freq_preds=[]
freq_truths=[]
phase_preds=[]
phase_truths=[]
water_supressions=[]
on_offs=[]
transient_model_snrs=[]
transient_models=[]

for model_snr in ["10"]:#,"5","10","x"]:
    for wsup in ["unsup"]:#["sup","unsup"]:
        off_fids = np.load(f"root/data/noisy_datasets/test_datasets/off_fids_{wsup}_{model_snr}.npy")[:1,:,:5]
        on_fids = np.load(f"root/data/noisy_datasets/test_datasets/on_fids_{wsup}_{model_snr}.npy")[:1,:,:5]
        ppm = np.load(f"root/data/noisy_datasets/test_datasets/ppm_{wsup}_{model_snr}.npy")
        t = np.load(f"root/data/noisy_datasets/test_datasets/t_{wsup}_{model_snr}.npy")
        
        y_freq = np.load(f"root/data/noisy_datasets/test_datasets/y_dataFreq_{wsup}_{model_snr}.npy")[:1,:,:,:5]
        y_phase = np.load(f"root/data/noisy_datasets/test_datasets/y_dataPhase_{wsup}_{model_snr}.npy")[:1,:,:,:5]

        raw_off_fids = np.expand_dims(off_fids,axis=2)
        raw_on_fids = np.expand_dims(on_fids,axis=2)

        raw_fids = np.concatenate([raw_off_fids,raw_on_fids],axis=2)

        print(raw_fids.shape)

        #print(ppm.shape)
        #print(t.shape)
        #print(raw_fids.shape)
        #print(y_freq.shape)



        for model in [complex_info,control_ma_info,control_tapper_info]:

            pipe_off = model["pipe"]()
            pipe_off.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            off_fids,off_freq_pred,off_phase_pred = apply_fpc(raw_fids[:,:,:1,:],pipe_off,t,512)

            pipe_on = model["pipe"]()
            pipe_on.load_model_from_weights(f"root/models/saved_weights/{model['name']}_freq_{wsup}_{model_snr}_{training_run}.h5",f"root/models/saved_weights/{model['name']}_phase_{wsup}_{model_snr}_{training_run}.h5")
            on_fids,on_freq_pred,on_phase_pred = apply_fpc(raw_fids[:,:,1:,:],pipe_on,t,512)


            off_y_freq = y_freq[:,:,0,:].flatten()
            on_y_freq = y_freq[:,:,1,:].flatten()
            off_y_phase = y_phase[:,:,0,:].flatten()
            on_y_phase = y_phase[:,:,1,:].flatten()

            print(on_y_freq)
            print(on_freq_pred)
            print("---------------------")


            transients+=list(range(1,6))*2*off_fids.shape[0]
            transient_scans+=list(np.repeat(np.arange(1,off_fids.shape[0]+1),10))
            freq_preds += list(off_freq_pred.flatten()) + list(on_freq_pred.flatten())
            phase_preds += list(off_phase_pred.flatten()) + list(on_phase_pred.flatten())
            freq_truths+= list(off_y_freq) + list(on_y_freq)
            phase_truths+= list(off_y_phase) + list(on_y_phase)

            if wsup=="sup":
                water_supression = True
            else:
                water_supression = False

            water_supressions+=[water_supression]*10*on_fids.shape[0]
            on_offs+=list(np.repeat(np.array([False,True]),5*on_fids.shape[0]))
            transient_model_snrs+=[model_snr]*10*on_fids.shape[0]
            transient_models+=[model["name"]]*10*on_fids.shape[0]
            



metrics_df = pd.DataFrame({
    "scan":scans,
    "model_snr":model_snrs,
    "model":models,
    "cho_std":cho_stds,
    "on_cr_fwhm":on_cr_fwhms,
    "off_cr_fwhm":off_cr_fwhms,
    "gaba_fwhm":gaba_fwhms,
    "on_cr_snr":on_cr_snrs,
    "off_cr_snr":off_cr_snrs,
    "gaba_snr":gaba_snrs
})

transient_metrics_df = pd.DataFrame({
    "transient":transients,
    "scan":transient_scans,
    "model_snr":transient_model_snrs,
    "model":transient_models,
    "water_supression":water_supressions,
    "on_off":on_offs,
    "freq_pred":freq_preds,
    "phase_pred":phase_preds,
    "true_freq":freq_truths,
    "true_phase":phase_truths
})

(1, 2048, 2, 5)
[20.60005213 -2.98294857 37.34242578 -5.05168956 23.2199231 ]
[[[[  5.654336   -11.311649    12.876937   -14.799534     0.74158365]]]]
---------------------
[20.60005213 -2.98294857 37.34242578 -5.05168956 23.2199231 ]
[[[[  5.681557   -11.458968    12.888055   -14.9064245    0.74361026]]]]
---------------------
[20.60005213 -2.98294857 37.34242578 -5.05168956 23.2199231 ]
[[[[  5.6865406 -11.474461   12.837241  -14.935702    0.7491906]]]]
---------------------
